<a href="https://colab.research.google.com/github/debrrox/impact_analysis/blob/read_emails/mentions2csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langdetect

In [ ]:
!python -m spacy download xx_ent_wiki_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 30.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import re
import os
import glob
from datetime import datetime
import email
from email import policy
from email.parser import BytesParser
from bs4 import BeautifulSoup, Comment
import pandas as pd
import json
import langdetect
import spacy
import numpy as np
import urllib
from urllib.parse import urlparse


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
COLUMNS = ['Title', 'Source', 'Summary', 'Date', 'State', 'Website', 'Language', 'NER', 'G/T']
SPACY_NLP = spacy.load("xx_ent_wiki_sm")

In [ ]:
def extract_links_from_google(eml_file_path):
    with open(eml_file_path, 'r') as file:
        # Parse the eml file
        eml_content = file.read()
    json_match = re.search(r'application/json">(.*?)</script>', eml_content, re.DOTALL)
    data = []
    if json_match:
        json_str = json_match.group(1)
        # Fix any escaped characters
        json_str = json_str.replace('=\n', '').replace('=3D', '=').replace('\n', '')
        # Load the JSON into a dictionary
        data_dict = json.loads(json_str)["cards"]
        refs = [elmt["widgets"] for elmt in data_dict]
        if refs:
            refs = refs[0]
        for ref in refs:
            title = ref["title"]
            parsed_url = urllib.parse.urlparse(ref["url"])
            # Extract the query parameters
            query_params = urllib.parse.parse_qs(parsed_url.query)
            # Get the real URL from the 'url' parameter
            source = query_params.get('url', [None])[0]
            summary = urllib.parse.unquote(ref["description"].replace('=', '%') , 'utf-8')
            website = urlparse(source).netloc.replace("www.", "")
            lang_sum = langdetect.detect(summary)
            ner = detect_NER(summary)
            # TODO figure out date
            # TODO Add state??
            data.append([title, source, summary, "", "", website, lang_sum, ner, "G"])
    # Create DataFrame
    df = pd.DataFrame(data, columns=COLUMNS)
    return df

In [ ]:
def extract_links_from_talkwalker(eml_file_path):
    with open(eml_file_path, 'rb') as file:
        # Parse the eml file
        msg = BytesParser(policy=policy.default).parse(file)
    html_content = msg.get_body(preferencelist=('plain', 'html')).get_content()
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the NEWS and TWITTER comments
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    news_index = None
    twitter_index = None

    for i, comment in enumerate(comments):
        if 'NEWS' in comment:
            news_index = i
        if 'TWITTER' in comment:
            twitter_index = i

    # Extract the content between NEWS and TWITTER
    data = []
    if news_index and twitter_index:
        news_content = comments[news_index].find_next_sibling()
        elements = []
        while news_content and news_content != comments[twitter_index]:
            elements.append(news_content)
            news_content = news_content.find_next_sibling()

        # Prepare data for DataFrame
        for element in elements:
            if element.name == 'tr':
                title = element.find('a').text if element.find('a') else None
                source = element.find('a')['href'] if element.find('a') else None
                summary = None
                date = None
                state = None
                website = None
                lang_sum = None
                ner = None
                for td in element.find_all('td'):
                    date_state_website = td.text.strip().split(' | ')
                    if td.text.strip()[:3]=="..."  or  td.text.strip()[-3:]=="..." :
                        summary = td.text.replace("\n", " ").strip()
                        lang_sum = langdetect.detect(summary)
                        ner = detect_NER(summary)
                    elif len(date_state_website) == 3:
                        if len(date_state_website[0]) < 15:
                            date = datetime.strptime(date_state_website[0], '%d.%m.%y %H:%M')
                            state = date_state_website[1]
                            website = date_state_website[2]
                if title and "alerts.talkwalker.com" not in source and date:
                    data.append([title, source, summary, date, state, website, lang_sum, ner, "T"])

    # Create DataFrame
    df = pd.DataFrame(data, columns=COLUMNS)
    return df

In [ ]:
def detect_NER(text):
    nlp = SPACY_NLP
    doc = nlp(text)
    for ent in doc.ents:
        if ent.text.lower() == "disclose":
            return ent.label_
    return None

In [ ]:
def remove_non_articles(df):
    non_none_cols = ['Title', 'Source', 'Summary', 'NER']
    df = df.dropna(subset=non_none_cols)
    df = df[~df["Website"].str.contains("disclose.ngo")]
    return df


In [ ]:
def save_csv(df, output_csv_path):
    """if os.path.exists(output_csv_path):
        df.to_csv(output_csv_path, mode='a', index=False, header=False)
    else:
        df.to_csv(output_csv_path, mode='w', index=False, header=True)
    """
    df.to_csv(output_csv_path, mode='w', index=False, header=True)


In [ ]:
def main():# Call the function and print the extracted links

    output_file = "/content/gdrive/MyDrive/Disclose/alerts.csv"
    if os.path.exists(output_file):
        os.remove(output_file)
        #df = pd.read_csv(output_file)
    #else:
    df = pd.DataFrame(columns=COLUMNS)
    input_folder = "/content/gdrive/MyDrive/Disclose/Emails"
    for eml_file_path in glob.glob(os.path.join(input_folder, "*")):
        if "talkwalker" in eml_file_path:
          df_new = extract_links_from_talkwalker(eml_file_path)
        else:
          df_new = extract_links_from_google(eml_file_path)
        df = pd.concat([df, df_new], ignore_index=True)
    df.drop_duplicates(inplace=True, ignore_index=True)
    df = remove_non_articles(df)
    save_csv(df, output_file)

In [ ]:
main()

<ipython-input-23-da326fa14894>:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_new], ignore_index=True)
